## Raghav Virmani  101803657  COE1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# **Question 1**

In [ ]:
X=np.array([i*np.pi/180 for i in range (60,300,4)])
np.random.seed(10)
y=np.sin(X)+np.random.normal(0, 0.15, len(X))
df= pd.DataFrame(np.column_stack([X,y]),columns=['X','y'])
for i in range (2,16):
  colname='X_%d'%i
  df[colname]=df['X']**i
# print(df)

In [ ]:
X=df.drop(['y'],axis=1).values
Y=df.iloc[:,1].values
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
X_scaled=np.insert(X_scaled,0,values=1,axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
betas=[]
l_rate = [0.0001, 0.001, 0.01, 0.1, 1, 10]
r_para = [pow(10,-15), pow(10,-10), pow(10,-5), pow(10,-3), 0, 1, 10, 20]
for learning_rate in l_rate:
  for r_p in r_para:
    beta = np.zeros(X_train.shape[1])
    for j in range(X_train.shape[1]):
      parsum=0
      for i in range(X_train.shape[0]):
        sum=0
        sum+=beta[0]
        for k in range(X_train.shape[1]):
          if k==0:
            continue
          sum+=beta[k]*X_train[i][k]
        sum-=Y_train[i]
        sum*=X_train[i][j]
      parsum=sum
      one=(parsum*learning_rate)/X_train.shape[0]
      two=1-((learning_rate*r_p)/X_train.shape[0])
      beta[j]=(beta[j]*two)-one
    betas.append(beta)

In [ ]:
r2 = []
from sklearn.metrics import r2_score
for beta in betas:
  Y_pred_val = X_val.dot(beta)
  r2.append(r2_score(Y_val, Y_pred_val))

In [ ]:
max_index = r2.index(max(r2))
optimal_beta = betas[max_index]
Y_pred_final = X_test.dot(optimal_beta)
r2_final = r2_score(Y_test, Y_pred_final)

In [ ]:
r2_final

0.7623215188142889

# **Question 2**

In [ ]:
df=pd.read_csv('/content/Hitters.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      322 non-null    int64  
 1   Hits       322 non-null    int64  
 2   HmRun      322 non-null    int64  
 3   Runs       322 non-null    int64  
 4   RBI        322 non-null    int64  
 5   Walks      322 non-null    int64  
 6   Years      322 non-null    int64  
 7   CAtBat     322 non-null    int64  
 8   CHits      322 non-null    int64  
 9   CHmRun     322 non-null    int64  
 10  CRuns      322 non-null    int64  
 11  CRBI       322 non-null    int64  
 12  CWalks     322 non-null    int64  
 13  League     322 non-null    object 
 14  Division   322 non-null    object 
 15  PutOuts    322 non-null    int64  
 16  Assists    322 non-null    int64  
 17  Errors     322 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  322 non-null    object 
dtypes: float64

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
AtBat,322.0,380.928571,153.404981,16.0,255.25,379.5,512.00,687.0
Hits,322.0,101.024845,46.454741,1.0,64.00,96.0,137.00,238.0
HmRun,322.0,10.770186,8.709037,0.0,4.00,8.0,16.00,40.0
Runs,322.0,50.909938,26.024095,0.0,30.25,48.0,69.00,130.0
RBI,322.0,48.027950,26.166895,0.0,28.00,44.0,64.75,121.0
Walks,322.0,38.742236,21.639327,0.0,22.00,35.0,53.00,105.0
Years,322.0,7.444099,4.926087,1.0,4.00,6.0,11.00,24.0
CAtBat,322.0,2648.683230,2324.205870,19.0,816.75,1928.0,3924.25,14053.0
CHits,322.0,717.571429,654.472627,4.0,209.00,508.0,1059.25,4256.0
CHmRun,322.0,69.490683,86.266061,0.0,14.00,37.5,90.00,548.0


In [ ]:
df[df.isnull().any(axis=1)].head(3)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
15,183,39,3,20,15,11,3,201,42,3,20,16,11,A,W,118,0,0,NaN,A
18,407,104,6,57,43,65,12,5233,1478,100,643,658,653,A,W,912,88,9,NaN,A


In [ ]:
df.isnull().sum().sum()

59

(a) Data Preprocessing

In [ ]:
df=df.copy()
df.corr()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
AtBat,1.000000,0.967939,0.592198,0.913060,0.820539,0.669845,0.047372,0.235526,0.252717,0.236659,0.266534,0.244053,0.166123,0.317550,0.353824,0.352117,0.394771
Hits,0.967939,1.000000,0.562158,0.922187,0.811073,0.641211,0.044767,0.227565,0.255815,0.202712,0.261787,0.232005,0.151818,0.310673,0.320455,0.310038,0.438675
HmRun,0.592198,0.562158,1.000000,0.650988,0.855122,0.481014,0.116318,0.221882,0.220627,0.493227,0.262361,0.351979,0.233154,0.282923,-0.106329,0.039318,0.343028
Runs,0.913060,0.922187,0.650988,1.000000,0.798206,0.732213,0.004541,0.186497,0.204830,0.227913,0.250556,0.205976,0.182168,0.279347,0.220567,0.240475,0.419859
RBI,0.820539,0.811073,0.855122,0.798206,1.000000,0.615997,0.146168,0.294688,0.308201,0.441771,0.323285,0.393184,0.250914,0.343186,0.106591,0.193370,0.449457
Walks,0.669845,0.641211,0.481014,0.732213,0.615997,1.000000,0.136475,0.277175,0.280671,0.332473,0.338478,0.308631,0.424507,0.299515,0.149656,0.129382,0.443867
Years,0.047372,0.044767,0.116318,0.004541,0.146168,0.136475,1.000000,0.920289,0.903631,0.726872,0.882877,0.868812,0.838533,-0.004684,-0.080638,-0.162140,0.400657
CAtBat,0.235526,0.227565,0.221882,0.186497,0.294688,0.277175,0.920289,1.000000,0.995063,0.798836,0.983345,0.949219,0.906501,0.062283,0.002038,-0.066922,0.526135
CHits,0.252717,0.255815,0.220627,0.204830,0.308201,0.280671,0.903631,0.995063,1.000000,0.783306,0.984609,0.945141,0.890954,0.076547,-0.002523,-0.062756,0.548910
CHmRun,0.236659,0.202712,0.493227,0.227913,0.441771,0.332473,0.726872,0.798836,0.783306,1.000000,0.820243,0.929484,0.799983,0.112724,-0.158511,-0.138115,0.524931


In [ ]:
df['Year_lab'] = pd.cut(x=df['Years'], bins=[0, 3, 6, 10, 15, 19, 24])
df.groupby(['League','Division', 'Year_lab']).agg({'Salary':'mean'})

Salary
League Division Year_lab            
A      E        (0, 3]    112.500000
                (3, 6]    655.568182
                (6, 10]   852.738125
                (10, 15]  816.311353
                (15, 19]  665.416750
                (19, 24]         NaN
       W        (0, 3]    153.613636
                (3, 6]    401.360000
                (6, 10]   633.958375
                (10, 15]  835.250000
                (15, 19]  479.000000
                (19, 24]  487.500000
N      E        (0, 3]    248.520813
                (3, 6]    501.191650
                (6, 10]   824.226143
                (10, 15]  894.322667
                (15, 19]  662.500000
                (19, 24]         NaN
       W        (0, 3]    191.766667
                (3, 6]    458.333333
                (6, 10]   563.229188
                (10, 15]  721.894000
                (15, 19]  760.833250
                (19, 24]  475.000000

In [ ]:
df['Salary'] = df.groupby(['League', 'Division', 'Year_lab'])['Salary'].transform(lambda x: x.fillna(x.mean()))
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague,Year_lab
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,112.5,A,"(0, 3]"
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N,"(10, 15]"
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A,"(0, 3]"
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N,"(10, 15]"
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N,"(0, 3]"


In [ ]:
df.isnull().sum()

AtBat        0
Hits         0
HmRun        0
Runs         0
RBI          0
Walks        0
Years        0
CAtBat       0
CHits        0
CHmRun       0
CRuns        0
CRBI         0
CWalks       0
League       0
Division     0
PutOuts      0
Assists      0
Errors       0
Salary       0
NewLeague    0
Year_lab     0
dtype: int64

In [ ]:
df.shape

(322, 21)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['League'] = le.fit_transform(df['League'])
df['Division'] = le.fit_transform(df['Division'])
df['NewLeague'] = le.fit_transform(df['NewLeague'])
df['Year_lab'] = le.fit_transform(df['Year_lab'])

In [ ]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague,Year_lab
0,293,66,1,30,29,14,1,293,66,1,30,29,14,0,0,446,33,20,112.5,0,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,1,1,632,43,10,475.0,1,3
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,0,1,880,82,14,480.0,0,0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,1,0,200,11,3,500.0,1,3
4,321,87,10,39,42,30,2,396,101,12,48,46,33,1,0,805,40,4,91.5,1,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      322 non-null    int64  
 1   Hits       322 non-null    int64  
 2   HmRun      322 non-null    int64  
 3   Runs       322 non-null    int64  
 4   RBI        322 non-null    int64  
 5   Walks      322 non-null    int64  
 6   Years      322 non-null    int64  
 7   CAtBat     322 non-null    int64  
 8   CHits      322 non-null    int64  
 9   CHmRun     322 non-null    int64  
 10  CRuns      322 non-null    int64  
 11  CRBI       322 non-null    int64  
 12  CWalks     322 non-null    int64  
 13  League     322 non-null    int64  
 14  Division   322 non-null    int64  
 15  PutOuts    322 non-null    int64  
 16  Assists    322 non-null    int64  
 17  Errors     322 non-null    int64  
 18  Salary     322 non-null    float64
 19  NewLeague  322 non-null    int64  
 20  Year_lab  

In [ ]:
from sklearn import preprocessing
df_X= df.drop(["Salary","League","Division","NewLeague"], axis=1)
scaled_cols5=preprocessing.normalize(df_X)
scaled_cols=pd.DataFrame(scaled_cols5, columns=df_X.columns)
scaled_cols.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Year_lab
0,0.472401,0.106411,0.001612,0.048369,0.046756,0.022572,0.001612,0.472401,0.106411,0.001612,0.048369,0.046756,0.022572,0.719082,0.053206,0.032246,0.000000
1,0.085657,0.022026,0.001903,0.006526,0.010333,0.010605,0.003807,0.937879,0.227060,0.018763,0.087289,0.112578,0.101973,0.171858,0.011693,0.002719,0.000816
2,0.237036,0.064331,0.008907,0.032660,0.035630,0.037609,0.001485,0.803645,0.226149,0.031176,0.110848,0.131631,0.130147,0.435473,0.040578,0.006928,0.000000
3,0.082624,0.023488,0.003332,0.010828,0.012993,0.006163,0.001832,0.937518,0.262365,0.037481,0.137929,0.139595,0.058970,0.033316,0.001832,0.000500,0.000500
4,0.331579,0.089867,0.010330,0.040285,0.043384,0.030989,0.002066,0.409050,0.104328,0.012395,0.049582,0.047516,0.034088,0.831529,0.041318,0.004132,0.000000


In [ ]:
cat_df=pd.concat([df.loc[:,"League":"Division"],df.loc[:,"NewLeague":"Year_lab"]], axis=1)
cat_df.head()

,League,Division,NewLeague,Year_lab
0,0,0,0,0
1,1,1,1,3
2,0,1,0,0
3,1,0,1,3
4,1,0,1,0


In [ ]:
df= pd.concat([scaled_cols,cat_df,df["Salary"]], axis=1)
print(df)
print(df.shape)

        AtBat      Hits     HmRun  ...  NewLeague  Year_lab  Salary
0    0.472401  0.106411  0.001612  ...          0         0   112.5
1    0.085657  0.022026  0.001903  ...          1         3   475.0
2    0.237036  0.064331  0.008907  ...          0         0   480.0
3    0.082624  0.023488  0.003332  ...          1         3   500.0
4    0.331579  0.089867  0.010330  ...          1         0    91.5
..        ...       ...       ...  ...        ...       ...     ...
317  0.169544  0.043324  0.002388  ...          1         1   700.0
318  0.083222  0.023004  0.000846  ...          0         3   875.0
319  0.256903  0.068147  0.001623  ...          0         1   385.0
320  0.155442  0.039064  0.002441  ...          0         2   960.0
321  0.120098  0.032356  0.001713  ...          0         3  1000.0

[322 rows x 22 columns]
(322, 22)


(b) Seperating Input and Output Features and Performing Scaling

In [ ]:
from sklearn.model_selection import train_test_split
X=df.drop("Salary", axis=1)
y=df["Salary"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

(c) Linear, Ridge and Lasso Regression

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
#Linear Regression
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
model = linreg.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(metrics.r2_score(y_test, y_pred))

0.376089582632248


In [ ]:
#Ridge Regression
from sklearn.linear_model import Ridge
ridreg = Ridge(alpha=0.5748, normalize=True)
model = ridreg.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(metrics.r2_score(y_test, y_pred))

-0.23721799488853912


In [ ]:
#Lasso Regression
from sklearn.linear_model import Lasso
lasreg = Lasso(alpha=0.5748, normalize=True)
model = lasreg.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(metrics.r2_score(y_test, y_pred))

0.26347337759310674


(d) According to R2 score, Linear Regression Model is performing the best as compared to Ridge Regression Model and Lasso Regression Model. 

# **Question 3**

In [ ]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()
dataset = pd.DataFrame(boston_dataset.data, columns = boston_dataset.feature_names)

In [ ]:
dataset.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [ ]:
dataset['MEDV'] = boston_dataset.target
dataset.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
X = dataset.iloc[:, 0:13].values
y = dataset.iloc[:, 13].values.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 25)
from sklearn.model_selection import cross_val_score

In [ ]:
#Ridge Regression Model
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

steps = [
    ('scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('model', Ridge(alpha=3.8, fit_intercept=True))
]

ridge_pipe = Pipeline(steps)
ridge_pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scalar',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('poly',
                 PolynomialFeatures(degree=2, include_bias=True,
                                    interaction_only=False, order='C')),
                ('model',
                 Ridge(alpha=3.8, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [ ]:
cv_ridge = cross_val_score(estimator = ridge_pipe, X = X_train, y = y_train.ravel(), cv = 10)
print('CV: ', cv_ridge.mean())

CV:  0.763562996213867


In [ ]:
#Lasso Regression Model
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

steps = [
    ('scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('model', Lasso(alpha=0.012, fit_intercept=True, max_iter=3000))
]

lasso_pipe = Pipeline(steps)
lasso_pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scalar',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('poly',
                 PolynomialFeatures(degree=2, include_bias=True,
                                    interaction_only=False, order='C')),
                ('model',
                 Lasso(alpha=0.012, copy_X=True, fit_intercept=True,
                       max_iter=3000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

In [ ]:
cv_lasso = cross_val_score(estimator = lasso_pipe, X = X_train, y = y_train, cv = 10)
print('CV: ', cv_lasso.mean())

CV:  0.7505443182491558
